# MCC185 Wigner Tomography Lab

This jupyter notebook has necessary functions to plot and manipulate the data. Feel free to consult them, and write to `helambe@chalmers.se` for any additional details. 

In [ ]:

def rotate_opt(
    data: np.ndarray, return_x: bool = False
) -> tuple[np.ndarray, Tuple[np.ndarray, float]]:
    """Rotates ``data`` so that all the signal is in the I quadrature (real part).

    Args:
        data: ``dtype`` should be ``complex128``.
        return_x: if :obj:`True`, return also the angle by which ``data`` was rotated.

    Returns:
        ``data * np.exp(1j * x)``, with ``x`` such that ``np.std(ret.imag)`` is minimum.
        ``dtype=complex128``.

        If ``return_x=True``, also return the angle ``x``.
    """
    # calculate the variance in steps of 1 deg
    N = 360
    _var = np.zeros(N)
    for ii in range(N):
        _data = data * np.exp(1j * 2 * np.pi / N * ii)
        _var[ii] = np.var(_data.imag)

    # the variance goes like cos(x)**2
    # FFT and find the phase at frequency "2"
    fft = np.fft.rfft(_var) / N
    # first solution
    x_fft1 = -np.angle(fft[2])  # compensate for measured phase
    x_fft1 -= np.pi  # we want to be at the minimum of cos(2x)
    x_fft1 /= 2  # move from frequency "2" to "1"
    # there's a second solution np.pi away (a minus sign)
    x_fft2 = x_fft1 + np.pi

    # convert to +/- interval
    x_fft1 = to_pm_pi(x_fft1)
    x_fft2 = to_pm_pi(x_fft2)
    # choose the closest to zero
    if np.abs(x_fft1) < np.abs(x_fft2):
        x_fft = x_fft1
    else:
        x_fft = x_fft2

    # rotate the data and return a copy
    data = data * np.exp(1j * x_fft)
    if return_x:
        return data, x_fft
    else:
        return data

def to_pm_pi(phase: float) -> float:
    """Converts a phase in radians into the [-π, +π) interval.

    Args:
        phase
    """
    return (phase + np.pi) % (2 * np.pi) - np.pi


# Resonator spectroscopy 

In [ ]:

x_axis = readout_freq
### SAVE FILE
if save_to_file:
    _dict = dict()
    for key in dir():
        if isinstance(globals()[key], (np.ndarray, float, int, str)):
            _dict[key] = globals()[key]

    np.savez(file_name[:-3], **_dict)

plt.figure()
plt.plot(x_axis, (demod_I_p **2 + demod_Q_p**2)**(1/2) * match_length, '.-')
plt.figure()
plt.plot(x_axis, np.arctan2(demod_Q_p,demod_I_p), '.-')

# Qubit spectroscopy 


In [ ]:
demod_I_p = np.array(demod_I_p)
demod_Q_p = np.array(demod_Q_p)


x_axis = control_freq
### SAVE FILE
if save_to_file:
    _dict = dict()
    for key in dir():
        if isinstance(globals()[key], (np.ndarray, float, int, str)):
            _dict[key] = globals()[key]

    np.savez(file_name[:-3], **_dict)

plt.figure()
plt.plot(x_axis, (demod_I_p **2 + demod_Q_p**2)**(1/2) * match_length, '.-')

# Rabi measurement 


In [ ]:
## plotting
xdata = control_amp

plt.figure()
# plt.plot(control_amp, demod_I_p * match_length, '.')
# plt.plot(control_amp, demod_Q_p * match_length, '.')
data, angle = rotate_opt(demod_I_p + 1j * demod_Q_p, True)
data = np.exp(1j * (angle)) * (demod_I_p + 1j * demod_Q_p)
plt.plot(xdata, np.real(data).T,'-o')
# print(angle)
if use_template_matching:
	plt.figure()
	data_matched = rotate_opt(avg_match_I_p + 1j * avg_match_Q_p)
	plt.plot(xdata, np.real(data_matched))

if use_fitting:
	import scipy.optimize as opt


	def func(t, A, T, phi, B):
		return A * np.sin(2 * np.pi * t / T + phi) + B


	ydata = np.real(data).T
	ydata = ydata[:,nr1-1]
	p0 = [(np.max(ydata) - np.min(ydata)), np.abs(2 * (xdata[np.argmax(ydata)] - xdata[np.argmin(ydata)])), -np.pi / 2,
		  (np.max(ydata) - np.min(ydata)) / 2]
	popt, pcov = opt.curve_fit(func, control_amp, ydata, p0=p0,
							   bounds=([-np.inf, 0, -np.pi, -np.inf], [np.inf, np.inf, np.pi, np.inf]))
	plt.plot(control_amp, func(control_amp, *popt))
	plt.vlines(0.5 * (popt[1]) - (popt[2] + np.pi / 2) / (2 * np.pi) * (popt[1]) + (nr1-1)*popt[1], np.min(ydata), np.max(ydata),
			   colors='k', linestyles='dashed',
			   label=r'$\pi=%.4f $' % (0.5 * (popt[1]) - (popt[2] + np.pi / 2) / (2 * np.pi) * (popt[1]) + (nr1-1)*popt[1]))
	plt.legend()

# plt.plot(control_amp, avg_match_I_p, '.')
# plt.plot(control_amp, avg_match_Q_p, '.')

plt.show()

# Ramsey measurement 


In [ ]:
## plotting
xdata = dt

plt.figure()
# plt.plot(control_amp, demod_I_p * match_length, '.')
# plt.plot(control_amp, demod_Q_p * match_length, '.')
data, angle = rotate_opt(demod_I_p + 1j * demod_Q_p, True)
plt.plot(xdata, np.real(data))


if use_fitting:
	import scipy.optimize as opt


	def func(t, A, f, T2, phi, B):
		return A * np.exp(-t / T2) * np.cos(2 * np.pi * t * f + phi) + B


	ydata = np.real(data)
	t = dt * 1e6
	A = max(ydata) - min(ydata)
	B = np.mean(ydata)
	ff = np.fft.fftfreq(len(t), (t[1] - t[0]))  # assume uniform spacing
	Fyy = abs(np.fft.fft(ydata))
	f = abs(ff[np.argmax(Fyy[1:]) + 1])
	phi = 0
	T2 = 20
	# p0 = [np.max(ydata)-np.min(ydata), (t[np.argmax(ydata)]-t[np.argmin(ydata)]), 20, 0, np.mean(ydata)]
	popt, pcov = opt.curve_fit(func, t, ydata, p0=[A, f, T2, phi, B])
	plt.plot(xdata, func(t, *popt), label=r'Freq=%.0f ' % (popt[1] * 1e6) + '\n' + r'T2=%.4f ' % (popt[2]))
	plt.legend()

# plt.plot(control_amp, avg_match_I_p, '.')
# plt.plot(control_amp, avg_match_Q_p, '.')

plt.show()

# T1 measurement 

In [ ]:
match_length = 1.6e-6  # s

## plotting
xdata = time_LUT

plt.figure()
data,angle = rotate_opt(demod_I_p * match_length + 1j*demod_Q_p * match_length,True)
plt.gca().ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.plot(xdata, np.real(data))
#plt.legend([r'$T_1=%.2e $' % (popt[1])])
plt.gca().ticklabel_format(style='sci', axis='x', scilimits=(0,0))
plt.show()



if use_fitting:
    import scipy.optimize as opt

    def func(t, A, T, B):
        return A*np.exp(-t/T) + B
    ydata = np.real(data)
    p0 = [(np.max(ydata)-np.min(ydata)), 20e-6, np.min(ydata)]
    popt, pcov = opt.curve_fit(func, xdata, ydata, p0=p0)
    plt.plot(xdata, func(xdata, *popt))
    plt.legend([r'$T_1=%.2e $'%(popt[1])])



# Cavity search 


In [ ]:
x_axis = disp_frequency

plt.figure()
data = rotate_opt(demod_I_p+1j*demod_Q_p)
plt.plot(x_axis, np.real(data.T))

# Displacement Spectroscopy 

I have added older data sets for this measurement, which might be easier to fit and calculate for $\chi$

In [ ]:
x_axis = control_freq

plt.figure()
#plt.plot(x_axis, demod_I_p * match_length, '.')
#plt.plot(control_amp, demod_Q_p * match_length, '.')
data,angle = rotate_opt(demod_I_p+1j*demod_Q_p, True)
plt.plot(x_axis, np.real(data.T))
